In [1]:
from tqdm import tqdm
import json
import requests
import geopandas as gpd
import geocoder
import pandas as pd
import numpy as np
from shapely.geometry import LineString, Point, Polygon
from func import *

In [2]:
specified_order = [
    '北投區', '士林區', '內湖區', '南港區',
    '松山區', '信義區', '中山區', '大同區',
    '中正區', '萬華區', '大安區', '文山區'
]

In [3]:
clinic_list_original = get_datataipei_api('3a02af7d-8c33-46c1-8226-c12a11610f6b')
hospital_list_original = get_datataipei_api('04a3d195-ee97-467a-b066-e471ff99d15d')
clinic_list = clinic_list_original.copy()
hospital_list = hospital_list_original.copy()
clinic_list

,_id,_importdate,機構名稱,地址,x,y
0,1,"{'date': '2023-07-21 14:52:19.943057', 'timezo...",佳譽堂中醫診所,臺北市中正區寧波西街71號,121.5169985,25.03048056
1,2,"{'date': '2023-07-21 14:52:19.953674', 'timezo...",仲景中醫診所,臺北市中正區忠孝東路2段16號2樓,121.5273294,25.04362777
2,3,"{'date': '2023-07-21 14:52:19.954972', 'timezo...",華御中醫診所,臺北市中正區濟南路2段51號,121.5310508,25.04047975
3,4,"{'date': '2023-07-21 14:52:19.956282', 'timezo...",日明耀中醫診所,臺北市中正區重慶南路一段66之1號2樓之1,121.5130979,25.04360909
4,5,"{'date': '2023-07-21 14:52:19.957350', 'timezo...",華陽中醫診所,臺北市中正區新生南路一段16之4號,121.5324342,25.04321262
...,...,...,...,...,...,...
3616,3617,"{'date': '2023-07-21 14:52:24.306041', 'timezo...",美學辰安牙醫診所,臺北市文山區指南路一段18號,121.5692076,24.98753838
3617,3618,"{'date': '2023-07-21 14:52:24.307228', 'timezo...",實踐牙醫診所,臺北市文山區忠順街一段１４３號,121.5608534,24.98444815
3618,3619,"{'date': '2023-07-21 14:52:24.308553', 'timezo...",慶聯牙醫診所,臺北市文山區木柵路三段１３６號１樓,121.5681917,24.98876049
3619,3620,"{'date': '2023-07-21 14:52:24.309801', 'timezo...",北持牙醫診所,臺北市文山區木柵路一段267號,121.5533959,24.98812263


In [4]:
ob_gyn_original = get_datataipei_api('ba860191-ad03-49a3-8df4-29e0a01f3026')
ob_gyn_copy = ob_gyn_original.copy()
ob_gyn_address = ob_gyn_copy[["機構名稱"]]
ob_gyn_address

,機構名稱
0,醫療財團法人徐元智先生醫藥基金會遠東聯合診所
1,祈新婦產科診所
2,江恩婦產科診所
3,小禾馨懷寧小兒專科診所
4,國立台灣大學醫學院附設醫院
...,...
138,臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理
139,愛文診所
140,仁荔健康診所
141,愛文婦產科診所


In [ ]:
ob_gyn_dist = ob_gyn_copy[["機構名稱"]]

In [ ]:
addr_info = geocoder.arcgis("台北市-吳輝明婦產科診所") # key="Ai-9ZhTsh1MEdbfSsTFrEIphmhjZockeOt0C2_IyG572ixNQyvPhksZuJep8agCx"
addr_info.json

In [6]:
def getCoordinateOfHospital(name):
    try:
        name_tmp = name
        if(name_tmp == "臺北市立聯合醫院附設大同門診部"):
            name_tmp = "臺北市立聯合醫院"
        if("醫院" in name_tmp):
            x = hospital_list.loc[hospital_list['機構名稱'] == name_tmp]["x"].reset_index(drop=True)[0]
            y = hospital_list.loc[hospital_list['機構名稱'] == name_tmp]["y"].reset_index(drop=True)[0]
        else:
            x = clinic_list.loc[clinic_list['機構名稱'] == name_tmp]["x"].reset_index(drop=True)[0]
            y = clinic_list.loc[clinic_list['機構名稱'] == name_tmp]["y"].reset_index(drop=True)[0]
        return Point(x, y)
    except:
        return 'NA'

In [12]:
def sortHospital(name):
    if("醫院" in name):
        return "醫院"
    else:
        return "診所"

In [9]:
def getDistrictOfHospital(name):
    try:
        name_tmp = name
        if(name_tmp == "臺北市立聯合醫院附設大同門診部"):
            name_tmp = "臺北市立聯合醫院"
        if("醫院" in name_tmp):
            address = hospital_list.loc[hospital_list['機構名稱'] == name_tmp]["地址"].reset_index(drop=True)[0]
        else:
            address = clinic_list.loc[clinic_list['機構名稱'] == name_tmp]["地址"].reset_index(drop=True)[0]
        idx = address.index('區')
        return address[idx-2:idx+1]
    except:
        # if(name_tmp[:3]!="台北市" or name_tmp[:3]!="臺北市"):
        #     name_tmp = "台北市" + name_tmp
        # addr_info = geocoder.arcgis(name_tmp) # address to coordinate
        # addr_json = addr_info.json
        # # print(addr_json)
        # if(addr_json != None):
        #     if(addr_json['score']>=90):
        #         return Point(
        #             addr_json['lng'],
        #             addr_json['lat']
        #         )
        # print(name_tmp)
        return 'NA'

In [ ]:
getDistrictOfHospital("臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理")

'文山區'

In [ ]:
ob_gyn_dist['type'] = ob_gyn_dist['機構名稱'].apply(
    sortHospital
)
ob_gyn_dist

C:\Users\10413\AppData\Local\Temp\ipykernel_14616\3927440527.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ob_gyn_dist['type'] = ob_gyn_dist['機構名稱'].apply(


,機構名稱,type
0,醫療財團法人徐元智先生醫藥基金會遠東聯合診所,診所
1,祈新婦產科診所,診所
2,江恩婦產科診所,診所
3,小禾馨懷寧小兒專科診所,診所
4,國立台灣大學醫學院附設醫院,醫院
...,...,...
138,臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理,醫院
139,愛文診所,診所
140,仁荔健康診所,診所
141,愛文婦產科診所,診所


In [ ]:
ob_gyn_dist['district'] = ob_gyn_dist['機構名稱'].apply(
    getDistrictOfHospital
)
ob_gyn_dist

台北市聖心婦家醫診所
台北市繼承婦產科診所
台北市瑞麗嘉健康美學慶中診所
台北市楊鵬生婦產科診所
台北市宜蘊婦產科診所
台北市鄢源貴婦產科診所
台北市劉教授婦產科診所
台北市愛文婦產科診所


C:\Users\10413\AppData\Local\Temp\ipykernel_14616\244576959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ob_gyn_dist['district'] = ob_gyn_dist['機構名稱'].apply(


,機構名稱,type,district
0,醫療財團法人徐元智先生醫藥基金會遠東聯合診所,診所,中正區
1,祈新婦產科診所,診所,中正區
2,江恩婦產科診所,診所,中正區
3,小禾馨懷寧小兒專科診所,診所,中正區
4,國立台灣大學醫學院附設醫院,醫院,中正區
...,...,...,...
138,臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理,醫院,文山區
139,愛文診所,診所,文山區
140,仁荔健康診所,診所,文山區
141,愛文婦產科診所,診所,NA


In [14]:
missing_hospital_list = ["聖心婦家醫診所", "繼承婦產科診所", "瑞麗嘉健康美學慶中診所", "楊鵬生婦產科診所", "宜蘊婦產科診所", "鄢源貴婦產科診所", "劉教授婦產科診所", "美迪婦產科診所"]
missing_dist_list = ["大同區", "中山區", "松山區", "大安區", "大安區", "大安區", "信義區", "信義區"]

In [16]:
def findIndexByName(df, name, column):
    return df.index[df[column] == name].tolist()



In [ ]:
for i in range(len(missing_hospital_list)):
    idx = findIndexByName(ob_gyn_dist, missing_hospital_list[i], "機構名稱")[0]
    ob_gyn_dist.at[idx,"district"] = missing_dist_list[i]

In [ ]:
idx = findIndexByName(ob_gyn_dist, "美迪婦產科診所", "機構名稱")[0]
idx

106

In [ ]:
ob_gyn_dist = ob_gyn_dist[ob_gyn_dist["district"] != 'NA'].reset_index(drop=True)

In [ ]:
grouped = ob_gyn_dist.groupby(ob_gyn_dist.type)
ob_gyn_dist_clinic = grouped.get_group("診所").reset_index(drop=True)
ob_gyn_dist_hospital = grouped.get_group("醫院").reset_index(drop=True)
print(ob_gyn_dist_clinic)
print(ob_gyn_dist_hospital)

                       機構名稱 type district
0    醫療財團法人徐元智先生醫藥基金會遠東聯合診所   診所      中正區
1                   祈新婦產科診所   診所      中正區
2                   江恩婦產科診所   診所      中正區
3               小禾馨懷寧小兒專科診所   診所      中正區
4                      吉星診所   診所      中正區
..                      ...  ...      ...
111                 天慈婦產科診所   診所      南港區
112              景美蔡登元婦產科診所   診所      文山區
113                    愛文診所   診所      文山區
114                  仁荔健康診所   診所      文山區
115                 長欣婦產科診所   診所      文山區

[116 rows x 3 columns]
                       機構名稱 type district
0             國立台灣大學醫學院附設醫院   醫院      中正區
1                  臺北市立聯合醫院   醫院      大同區
2           臺北市立聯合醫院附設大同門診部   醫院      大同區
3    台灣基督長老教會馬偕醫療財團法人馬偕紀念醫院   醫院      中山區
4                    協和婦女醫院   醫院      中山區
5        三軍總醫院松山分院附設民眾診療服務處   醫院      松山區
6          長庚醫療財團法人台北長庚紀念醫院   醫院      松山區
7        基督復臨安息日會醫療財團法人臺安醫院   醫院      松山區
8                    博仁綜合醫院   醫院      松山區
9              中山醫療社團法人中山醫院   醫院      大安區
10        

In [ ]:
clinic_dist_count = ob_gyn_dist_clinic['district'].value_counts().astype('int')
hospital_dist_count = ob_gyn_dist_hospital['district'].value_counts().astype('int')

In [ ]:
for dist in specified_order:
    if dist not in clinic_dist_count:
        clinic_dist_count[dist] = 0
    if dist not in hospital_dist_count:
        hospital_dist_count[dist] = 0
hospital_dist_count

district
大安區    5
松山區    4
萬華區    3
北投區    3
內湖區    3
大同區    2
中山區    2
中正區    1
信義區    1
士林區    1
文山區    1
南港區    0
Name: count, dtype: int32

In [ ]:
results = {
    'data': [
        {
            'name': "診所",
            'data': [
                int(clinic_dist_count[dist]) for dist in specified_order
            ]
        },
        {
            'name': "醫院",
            'data': [
                int(hospital_dist_count[dist]) for dist in specified_order
            ]
        }
    ]
}

In [ ]:
with open('4000.json', "w") as json_file:
    json.dump(results, json_file, indent=4, ensure_ascii=False)  

In [ ]:
result = ob_gyn_dist.to_json(orient="records")
parsed = json.loads(result)
with open('4000.json', "w") as json_file:
    json.dump(parsed, json_file, indent=4, ensure_ascii=False)  

In [7]:
ob_gyn_address['geometry'] = ob_gyn_address['機構名稱'].apply(
    getCoordinateOfHospital
)
ob_gyn_address

C:\Users\Joachim Chen\AppData\Local\Temp\ipykernel_17204\1281934065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ob_gyn_address['geometry'] = ob_gyn_address['機構名稱'].apply(


,機構名稱,geometry
0,醫療財團法人徐元智先生醫藥基金會遠東聯合診所,POINT (121.5110354 25.04300291)
1,祈新婦產科診所,POINT (121.5139896 25.04310023)
2,江恩婦產科診所,POINT (121.5169585 25.04544025)
3,小禾馨懷寧小兒專科診所,POINT (121.5133326 25.04310036)
4,國立台灣大學醫學院附設醫院,POINT (121.5183428 25.03553921)
...,...,...
138,臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理,POINT (121.5583303 25.00030628)
139,愛文診所,POINT (121.5565238 25.00023339)
140,仁荔健康診所,POINT (121.5449553 24.99846549)
141,愛文婦產科診所,NA


In [8]:
def getAddressOfHospital(name):
    try:
        name_tmp = name
        if(name_tmp == "臺北市立聯合醫院附設大同門診部"):
            name_tmp = "臺北市立聯合醫院"
        if("醫院" in name_tmp):
            addr = hospital_list.loc[hospital_list['機構名稱'] == name_tmp]["地址"].reset_index(drop=True)[0]
        else:
            addr = clinic_list.loc[clinic_list['機構名稱'] == name_tmp]["地址"].reset_index(drop=True)[0]
        return addr
    except:
        return 'NA'

In [10]:
ob_gyn_address['address'] = ob_gyn_address['機構名稱'].apply(
    getAddressOfHospital
)
ob_gyn_address['name'] = ob_gyn_address['機構名稱']
ob_gyn_address['dist'] = ob_gyn_address['name'].apply(
	getDistrictOfHospital
)
ob_gyn_address.drop('機構名稱', axis=1, inplace=True)
ob_gyn_address

C:\Users\Joachim Chen\AppData\Local\Temp\ipykernel_17204\515465044.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ob_gyn_address['address'] = ob_gyn_address['機構名稱'].apply(


,geometry,address,name,dist
0,POINT (121.5110354 25.04300291),臺北市中正區永綏街8至14號,醫療財團法人徐元智先生醫藥基金會遠東聯合診所,中正區
1,POINT (121.5139896 25.04310023),臺北市中正區懷寧街78號9、10樓,祈新婦產科診所,中正區
2,POINT (121.5169585 25.04544025),臺北市中正區許昌街8號,江恩婦產科診所,中正區
3,POINT (121.5133326 25.04310036),臺北市中正區重慶南路一段99號2樓,小禾馨懷寧小兒專科診所,中正區
4,POINT (121.5183428 25.03553921),臺北市中正區中山南路７、８號；常德街１號,國立台灣大學醫學院附設醫院,中正區
...,...,...,...,...
138,POINT (121.5583303 25.00030628),臺北市文山區興隆路三段111號,臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理,文山區
139,POINT (121.5565238 25.00023339),臺北市文山區興隆路三段79巷3號,愛文診所,文山區
140,POINT (121.5449553 24.99846549),臺北市文山區興隆路一段258號,仁荔健康診所,文山區
141,NA,NA,愛文婦產科診所,NA


In [13]:
ob_gyn_address['main_type'] = ob_gyn_address['name'].apply(
    sortHospital
)

In [17]:
# missing_hospital_list = ["聖心婦家醫診所", "繼承婦產科診所", "瑞麗嘉健康美學慶中診所", "楊鵬生婦產科診所", "宜蘊婦產科診所", "鄢源貴婦產科診所", "劉教授婦產科診所", "美迪婦產科診所"]
missing_address_list = ["臺北市大同區保安街1之1號", "臺北市中山區松江路276號5樓", "臺北市松山區南京東路三段259號5樓", "臺北市大安區信義路二段72號2樓", "臺北市大安區信義路三段149號15樓", "臺北市大安區信義路四段199巷2號5樓之1", "臺北市信義區忠孝東路五段236巷32號1樓", "臺北市信義區信義路五段18號2樓"]
for i in range(len(missing_hospital_list)):
    idx = findIndexByName(ob_gyn_address, missing_hospital_list[i], "name")[0]
    ob_gyn_address.at[idx,"dist"] = missing_dist_list[i]
    ob_gyn_address.at[idx,"address"] = missing_address_list[i]
    addr_json = geocoder.arcgis(missing_address_list[i]).json
    ob_gyn_address.at[idx,"geometry"] = Point(addr_json['lng'], addr_json['lat'])

In [18]:
ob_gyn_address = ob_gyn_address[ob_gyn_address["address"] != 'NA'].reset_index(drop=True)

In [19]:
ob_gyn_address

,geometry,address,name,dist,main_type
0,POINT (121.5110354 25.04300291),臺北市中正區永綏街8至14號,醫療財團法人徐元智先生醫藥基金會遠東聯合診所,中正區,診所
1,POINT (121.5139896 25.04310023),臺北市中正區懷寧街78號9、10樓,祈新婦產科診所,中正區,診所
2,POINT (121.5169585 25.04544025),臺北市中正區許昌街8號,江恩婦產科診所,中正區,診所
3,POINT (121.5133326 25.04310036),臺北市中正區重慶南路一段99號2樓,小禾馨懷寧小兒專科診所,中正區,診所
4,POINT (121.5183428 25.03553921),臺北市中正區中山南路７、８號；常德街１號,國立台灣大學醫學院附設醫院,中正區,醫院
...,...,...,...,...,...
137,POINT (121.5494378 25.00082992),臺北市文山區興隆路二段１８９號,景美蔡登元婦產科診所,文山區,診所
138,POINT (121.5583303 25.00030628),臺北市文山區興隆路三段111號,臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理,文山區,醫院
139,POINT (121.5565238 25.00023339),臺北市文山區興隆路三段79巷3號,愛文診所,文山區,診所
140,POINT (121.5449553 24.99846549),臺北市文山區興隆路一段258號,仁荔健康診所,文山區,診所


In [ ]:
ob_gyn_address.to_pickle("ob_gyn_address.pkl")

In [9]:
ob_gyn_address = pd.read_pickle("ob_gyn_address.pkl")

In [5]:
ob_gyn_address

,機構名稱,geometry,address,name,main_type
0,醫療財團法人徐元智先生醫藥基金會遠東聯合診所,POINT (121.5110354 25.04300291),臺北市中正區永綏街8至14號,醫療財團法人徐元智先生醫藥基金會遠東聯合診所,診所
1,祈新婦產科診所,POINT (121.5139896 25.04310023),臺北市中正區懷寧街78號9、10樓,祈新婦產科診所,診所
2,江恩婦產科診所,POINT (121.5169585 25.04544025),臺北市中正區許昌街8號,江恩婦產科診所,診所
3,小禾馨懷寧小兒專科診所,POINT (121.5133326 25.04310036),臺北市中正區重慶南路一段99號2樓,小禾馨懷寧小兒專科診所,診所
4,國立台灣大學醫學院附設醫院,POINT (121.5183428 25.03553921),臺北市中正區中山南路７、８號；常德街１號,國立台灣大學醫學院附設醫院,醫院
...,...,...,...,...,...
137,景美蔡登元婦產科診所,POINT (121.5494378 25.00082992),臺北市文山區興隆路二段１８９號,景美蔡登元婦產科診所,診所
138,臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理,POINT (121.5583303 25.00030628),臺北市文山區興隆路三段111號,臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理,醫院
139,愛文診所,POINT (121.5565238 25.00023339),臺北市文山區興隆路三段79巷3號,愛文診所,診所
140,仁荔健康診所,POINT (121.5449553 24.99846549),臺北市文山區興隆路一段258號,仁荔健康診所,診所


In [10]:
ob_gyn_address = pd.DataFrame(ob_gyn_address)

In [11]:
ob_gyn_address

,機構名稱,geometry,address,name,main_type
0,醫療財團法人徐元智先生醫藥基金會遠東聯合診所,POINT (121.5110354 25.04300291),臺北市中正區永綏街8至14號,醫療財團法人徐元智先生醫藥基金會遠東聯合診所,診所
1,祈新婦產科診所,POINT (121.5139896 25.04310023),臺北市中正區懷寧街78號9、10樓,祈新婦產科診所,診所
2,江恩婦產科診所,POINT (121.5169585 25.04544025),臺北市中正區許昌街8號,江恩婦產科診所,診所
3,小禾馨懷寧小兒專科診所,POINT (121.5133326 25.04310036),臺北市中正區重慶南路一段99號2樓,小禾馨懷寧小兒專科診所,診所
4,國立台灣大學醫學院附設醫院,POINT (121.5183428 25.03553921),臺北市中正區中山南路７、８號；常德街１號,國立台灣大學醫學院附設醫院,醫院
...,...,...,...,...,...
137,景美蔡登元婦產科診所,POINT (121.5494378 25.00082992),臺北市文山區興隆路二段１８９號,景美蔡登元婦產科診所,診所
138,臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理,POINT (121.5583303 25.00030628),臺北市文山區興隆路三段111號,臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理,醫院
139,愛文診所,POINT (121.5565238 25.00023339),臺北市文山區興隆路三段79巷3號,愛文診所,診所
140,仁荔健康診所,POINT (121.5449553 24.99846549),臺北市文山區興隆路一段258號,仁荔健康診所,診所


In [12]:
ob_gyn_address = ob_gyn_address.drop("機構名稱", axis=1)

In [20]:
ob_gyn_address = ob_gyn_address.iloc[:, [4,3,2,1,0]]

In [21]:
ob_gyn_address_gdf = gpd.GeoDataFrame(ob_gyn_address)
ob_gyn_address_gdf 

,main_type,dist,name,address,geometry
0,診所,中正區,醫療財團法人徐元智先生醫藥基金會遠東聯合診所,臺北市中正區永綏街8至14號,POINT (121.51104 25.04300)
1,診所,中正區,祈新婦產科診所,臺北市中正區懷寧街78號9、10樓,POINT (121.51399 25.04310)
2,診所,中正區,江恩婦產科診所,臺北市中正區許昌街8號,POINT (121.51696 25.04544)
3,診所,中正區,小禾馨懷寧小兒專科診所,臺北市中正區重慶南路一段99號2樓,POINT (121.51333 25.04310)
4,醫院,中正區,國立台灣大學醫學院附設醫院,臺北市中正區中山南路７、８號；常德街１號,POINT (121.51834 25.03554)
...,...,...,...,...,...
137,診所,文山區,景美蔡登元婦產科診所,臺北市文山區興隆路二段１８９號,POINT (121.54944 25.00083)
138,醫院,文山區,臺北市立萬芳醫院－委託財團法人臺北醫學大學辦理,臺北市文山區興隆路三段111號,POINT (121.55833 25.00031)
139,診所,文山區,愛文診所,臺北市文山區興隆路三段79巷3號,POINT (121.55652 25.00023)
140,診所,文山區,仁荔健康診所,臺北市文山區興隆路一段258號,POINT (121.54496 24.99847)


In [22]:
ob_gyn_address_gdf.to_file(
    '4000.geojson',
    driver='GeoJSON'
)